# Day 1: Data Engineering & Exploratory Data Analysis

## Overview

This notebook covers:
1. Loading and merging all 9 Olist datasets
2. Creating time-based train/val/test split
3. Exploratory Data Analysis (on training set ONLY)
4. Missing value analysis and imputation
5. Outlier detection and handling
6. Schema validation

**Key Principles:**
- Split BEFORE EDA to prevent data leakage
- All statistics computed on training set only
- Transformers fit on train, applied to val/test

## 1. Setup & Imports

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path

# Our custom modules
from src.data_loader import (
    load_raw_data, parse_date_columns, 
    create_time_based_split, save_split_ids,
    merge_datasets, create_target_variables,
    apply_split, save_splits_parquet,
    load_splits_parquet
)

from src.eda import (
    get_basic_stats, analyze_column_types,
    analyze_classification_target, analyze_regression_target,
    analyze_missing_values, detect_outliers_iqr,
    analyze_correlations, analyze_temporal_patterns,
    analyze_categorical
)

from src.data_quality import (
    process_training_data, process_inference_data,
    save_artifacts, validate_dataframe
)

# Settings
pd.set_option('display.max_columns', 50)
plt.style.use('seaborn-v0_8-whitegrid')

print("Setup complete!")

## 2. Load Raw Data

Loading all 9 CSV files from the Olist Brazilian E-Commerce dataset.

In [ ]:
# Load all 9 datasets
datasets = load_raw_data('../data/raw/')

# Parse date columns
datasets = parse_date_columns(datasets)

In [ ]:
# Quick look at the orders table (our main table)
datasets['orders'].head()

## 3. Create Time-Based Split (CRITICAL!)

We split the data BEFORE any analysis to prevent data leakage.

**Strategy:** Time-based split
- Train: 70% oldest orders (Sept 2016 - April 2018)
- Val: 15% middle orders (April - June 2018)
- Test: 15% newest orders (June - Oct 2018)

In [ ]:
# Create time-based split
train_ids, val_ids, test_ids = create_time_based_split(
    datasets['orders'],
    train_ratio=0.70,
    val_ratio=0.15,
    test_ratio=0.15
)

# Verify no overlap
print(f"\nNo overlap verification:")
print(f"  Train n Val: {len(set(train_ids) & set(val_ids))} (should be 0)")
print(f"  Train n Test: {len(set(train_ids) & set(test_ids))} (should be 0)")
print(f"  Val n Test: {len(set(val_ids) & set(test_ids))} (should be 0)")

## 4. Merge All Datasets

In [ ]:
# Merge all datasets into one
merged = merge_datasets(datasets)

# Create target variables
merged = create_target_variables(merged)

print(f"\nFinal merged shape: {merged.shape}")

In [ ]:
# Apply splits
train_df, val_df, test_df = apply_split(merged, train_ids, val_ids, test_ids)

## 5. EDA on Training Set Only

**IMPORTANT:** From here on, we only analyze the training set!

In [ ]:
# Basic statistics
stats = get_basic_stats(train_df, "Training Set")

### 5.1 Target Variable Analysis

In [ ]:
# Classification target: is_satisfied
clf_analysis = analyze_classification_target(train_df, 'is_satisfied')

In [ ]:
# Regression target: delivery_days
reg_analysis = analyze_regression_target(train_df, 'delivery_days')

In [ ]:
# Visualize target distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Classification target
train_df['is_satisfied'].value_counts().plot(kind='bar', ax=axes[0], color=['salmon', 'lightgreen'])
axes[0].set_title('Customer Satisfaction Distribution')
axes[0].set_xlabel('Satisfied (1) vs Unsatisfied (0)')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=0)

# Regression target
train_df['delivery_days'].dropna().hist(bins=50, ax=axes[1], color='steelblue', edgecolor='white')
axes[1].axvline(train_df['delivery_days'].median(), color='red', linestyle='--', label=f'Median: {train_df["delivery_days"].median():.1f}')
axes[1].set_title('Delivery Days Distribution')
axes[1].set_xlabel('Days')
axes[1].set_ylabel('Count')
axes[1].legend()

plt.tight_layout()
plt.show()

### 5.2 Missing Value Analysis

In [ ]:
# Analyze missing values
missing_df = analyze_missing_values(train_df)

In [ ]:
# Visualize missing values
cols_with_missing = missing_df[missing_df['missing_count'] > 0].head(15)

plt.figure(figsize=(12, 6))
plt.barh(cols_with_missing['column'], cols_with_missing['missing_pct'], color='coral')
plt.xlabel('Missing %')
plt.title('Columns with Missing Values')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

### 5.3 Temporal Analysis

In [ ]:
# Temporal patterns
temporal = analyze_temporal_patterns(train_df)

In [ ]:
# Orders over time
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Monthly trend
monthly = train_df.groupby(train_df['order_purchase_timestamp'].dt.to_period('M')).size()
monthly.plot(ax=axes[0], marker='o', color='steelblue')
axes[0].set_title('Orders Over Time (Monthly)')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Number of Orders')
axes[0].tick_params(axis='x', rotation=45)

# Day of week
dow_counts = train_df['order_purchase_timestamp'].dt.dayofweek.value_counts().sort_index()
dow_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
dow_counts.index = dow_names
dow_counts.plot(kind='bar', ax=axes[1], color='lightgreen', edgecolor='darkgreen')
axes[1].set_title('Orders by Day of Week')
axes[1].set_xlabel('Day')
axes[1].set_ylabel('Number of Orders')
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

### 5.4 Geographic Analysis

In [ ]:
# Customer states
state_analysis = analyze_categorical(train_df, 'customer_state')

In [ ]:
# Top 10 states
plt.figure(figsize=(10, 6))
train_df['customer_state'].value_counts().head(10).plot(kind='bar', color='teal', edgecolor='white')
plt.title('Orders by Customer State (Top 10)')
plt.xlabel('State')
plt.ylabel('Number of Orders')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

### 5.5 Product Analysis

In [ ]:
# Product categories
category_analysis = analyze_categorical(train_df, 'product_category_name_english')

In [ ]:
# Top 15 categories
plt.figure(figsize=(12, 6))
train_df['product_category_name_english'].value_counts().head(15).plot(kind='barh', color='steelblue')
plt.title('Top 15 Product Categories')
plt.xlabel('Number of Orders')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

### 5.6 Correlation Analysis

In [ ]:
# Correlations with satisfaction
is_satisfied_corr = analyze_correlations(train_df, target_col='is_satisfied')

In [ ]:
# Correlations with delivery time
delivery_corr = analyze_correlations(train_df, target_col='delivery_days')

In [ ]:
# Correlation heatmap
numeric_cols = ['price', 'freight_value', 'payment_value', 'delivery_days', 
                'delivery_delay_days', 'is_late_delivery', 'review_score', 'is_satisfied']
corr_matrix = train_df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0, fmt='.2f', square=True)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

### 5.7 Outlier Detection

In [ ]:
# Detect outliers
key_cols = ['price', 'freight_value', 'payment_value', 'delivery_days', 'product_weight_g']
outliers_df = detect_outliers_iqr(train_df, columns=key_cols)

In [ ]:
# Box plots for key numerical features
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for i, col in enumerate(['price', 'freight_value', 'delivery_days', 'product_weight_g']):
    train_df[col].dropna().plot(kind='box', ax=axes[i])
    axes[i].set_title(f'{col}')

plt.suptitle('Box Plots - Outlier Visualization', y=1.02)
plt.tight_layout()
plt.show()

## 6. Data Processing

Apply missing value imputation and outlier handling.

In [ ]:
# Process training data (fits transformers)
train_processed, artifacts = process_training_data(train_df)

# Save artifacts
save_artifacts(artifacts, '../models/data_processing_artifacts.json')

In [ ]:
# Process val and test using training artifacts
val_processed = process_inference_data(val_df, artifacts)
test_processed = process_inference_data(test_df, artifacts)

In [ ]:
# Validate all datasets
validate_dataframe(train_processed, 'Train')
validate_dataframe(val_processed, 'Val')
validate_dataframe(test_processed, 'Test')

## 7. Save Processed Data

In [ ]:
# Save processed datasets
train_processed.to_parquet('../data/processed/train_processed.parquet', index=False)
val_processed.to_parquet('../data/processed/val_processed.parquet', index=False)
test_processed.to_parquet('../data/processed/test_processed.parquet', index=False)

print("Saved all processed datasets!")

## 8. Summary

### Key Findings:

**Target Variables:**
- Classification: 72.5% satisfied, 27.5% unsatisfied (reasonably balanced)
- Regression: Mean 13.7 days delivery, heavily right-skewed

**Key Predictors of Satisfaction:**
- Late delivery is the #1 driver of dissatisfaction (-0.30 correlation)
- Delivery time strongly affects satisfaction (-0.29 correlation)

**Geographic Insights:**
- Sao Paulo (SP) accounts for 40% of orders
- RJ and MG are #2 and #3

**Product Insights:**
- bed_bath_table, sports_leisure, furniture_decor are top categories
- Credit card is dominant payment method (76%)

### Files Created:
- `data/splits/train.parquet`, `val.parquet`, `test.parquet`
- `data/processed/train_processed.parquet`, etc.
- `models/data_processing_artifacts.json`
- `data/processed/eda_statistics.json`

## Next Steps: Day 2

In the next notebook, we will:
1. Create sklearn pipelines for feature engineering
2. Build RFM, temporal, geographic features
3. Implement customer segmentation (clustering)
4. Add cluster labels as features for supervised learning